# Setup

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
from scipy.special import logit

# Read and preprocess the data
data = pd.read_csv("data/games_e1.csv")

In [2]:
data = pd.read_csv("data/games_e1.csv")
data["scenario"] = data["scenario"].astype("category")
data["mental_state"] = data["mental_state"].astype("category")
data.loc[data["participant_id"].isin([1, 82]), "participant_id"] = np.nan
data["participant_id"] = data["participant_id"].astype("category")
data = data.loc[(data["add_hint"] == False)]

# Descriptive

## Summary

In [3]:
print(data.info())
print(data.describe(include="all"))

<class 'pandas.core.frame.DataFrame'>
Index: 1204 entries, 0 to 1203
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   persuader_type      1204 non-null   object  
 1   mental_state        1204 non-null   category
 2   participant_id      1001 non-null   category
 3   persuader_role      1204 non-null   object  
 4   scenario            1204 non-null   category
 5   trial_num           1204 non-null   int64   
 6   success             1204 non-null   int64   
 7   persuasion_success  1204 non-null   int64   
 8   appeals_success     1204 non-null   int64   
 9   add_hint            1204 non-null   bool    
dtypes: bool(1), category(3), int64(4), object(2)
memory usage: 77.4+ KB
None
       persuader_type mental_state  participant_id     persuader_role  \
count            1204         1204          1001.0               1204   
unique              3            2           170.0                  1   
t

## Number of games per condition

In [4]:
n_games_per_condition = (
    data.groupby(["persuader_type", "mental_state"]).size().reset_index(name="n_games")
)
print(n_games_per_condition)

          persuader_type mental_state  n_games
0      gpt-4o-2024-11-20    no_reveal      200
1      gpt-4o-2024-11-20       reveal      200
2                  human    no_reveal      202
3                  human       reveal      202
4  o1-preview-2024-09-12    no_reveal      200
5  o1-preview-2024-09-12       reveal      200


/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_50401/3089819761.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["persuader_type", "mental_state"]).size().reset_index(name="n_games")


## Number of participants per condition

In [5]:
n_ppts_per_condition = (
    data.groupby(["persuader_type", "mental_state"])["participant_id"]
    .nunique()
    .reset_index(name="n_ppts")
)
print(n_ppts_per_condition)

          persuader_type mental_state  n_ppts
0      gpt-4o-2024-11-20    no_reveal       1
1      gpt-4o-2024-11-20       reveal       0
2                  human    no_reveal      79
3                  human       reveal      88
4  o1-preview-2024-09-12    no_reveal       1
5  o1-preview-2024-09-12       reveal       1


/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_50401/3796846038.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["persuader_type", "mental_state"])["participant_id"]


## Results

In [6]:
results = (
    data.groupby(["persuader_type", "mental_state"])
    .agg(
        n_ppts=("participant_id", pd.Series.nunique),
        success_rate=("success", lambda x: round(x.mean(), 3)),
    )
    .reset_index()
)
print(results)

          persuader_type mental_state  n_ppts  success_rate
0      gpt-4o-2024-11-20    no_reveal       1         0.090
1      gpt-4o-2024-11-20       reveal       0         0.175
2                  human    no_reveal      79         0.292
3                  human       reveal      88         0.371
4  o1-preview-2024-09-12    no_reveal       1         0.175
5  o1-preview-2024-09-12       reveal       1         0.795


/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_50401/3885407465.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  data.groupby(["persuader_type", "mental_state"])


# Jared: I'm not sure if any of the models are a correct translation from R. Just had o1 do it all with some revisions.

# Hypotheses Testing

We'll now use the `BinomialBayesMixedGLM` class from `statsmodels` to fit mixed-effects logistic regression models with random intercepts, similar to the `glmer` function in R's `lme4` package.

**Note:** The `BinomialBayesMixedGLM` fits the model using Bayesian estimation methods (Laplace approximation or variational Bayes), but we can interpret the results similarly to maximum likelihood estimation in frequentist approaches.

# H1

## Simple GLM

For H1 we’re using a logistic regression to ask “Is the human success rate in the `no_reveal` condition significantly greater than 0.1”?

Humans are significantly above chance on the `no_reveal` condition(!)

In [7]:
chance_prob = 0.1
chance_logodds = logit(chance_prob)

model_data = data.loc[
    (data["persuader_type"] == "human") & (data["mental_state"] == "no_reveal")
].copy()

# Add offset column
model_data["offset_term"] = chance_logodds

# Fit the GLM with offset
model = smf.glm(
    formula="success ~ 1",
    data=model_data,
    family=sm.families.Binomial(),
    offset=model_data["offset_term"],
)

result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                  202
Model:                            GLM   Df Residuals:                      201
Model Family:                Binomial   Df Model:                            0
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -122.01
Date:                Fri, 31 Jan 2025   Deviance:                       244.02
Time:                        08:22:37   Pearson chi2:                     202.
No. Iterations:                     4   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3119      0.155      8.479      0.0

/Users/jared/workspace/mindgames/env-mindgames/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/Users/jared/workspace/mindgames/env-mindgames/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: divide by zero encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
/Users/jared/workspace/mindgames/env-mindgames/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: invalid value encountered in multiply
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +


## Mixed Effects Model

We now fit a mixed-effects logistic regression with a random intercept for `scenario`.

In [8]:
# Prepare the data
model_data = data.loc[
    (data["persuader_type"] == "human") & (data["mental_state"] == "no_reveal")
].copy()

# Ensure 'scenario' is of type string for proper handling in formulas
model_data["scenario"] = model_data["scenario"].astype(str)

# Define the fixed effects formula (only intercept in this case)
formula = "success ~ 1"

# Define variance components using vc_formulas
vc_formula = {"scenario": "0 + C(scenario)"}

# Fit the model using the from_formula method
model = BinomialBayesMixedGLM.from_formula(
    formula=formula, vc_formulas=vc_formula, data=model_data
)

# Fit the model using variational Bayes method
result = model.fit_vb()

# Output the summary
print(result.summary())

               Binomial Mixed GLM Results
          Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
--------------------------------------------------------
Intercept    M    -0.8885   0.1544                      
scenario     V    -1.2772   0.3441 0.279   0.140   0.555
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


# H2

## Simple GLM

For H2 we're asking "Do human participants perform significantly better in `reveal` vs `no_reveal`".

The effect is directionally positive but not significant even in the simple GLM.

In [9]:
model_data = data.loc[data["persuader_type"] == "human"]

model = smf.glm(
    formula="success ~ mental_state", data=model_data, family=sm.families.Binomial()
)

result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                  404
Model:                            GLM   Df Residuals:                      402
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -255.26
Date:                Fri, 31 Jan 2025   Deviance:                       510.51
Time:                        08:22:37   Pearson chi2:                     404.
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007063
Covariance Type:            nonrobust                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0

## Mixed Effects Model

In [10]:
# Prepare the data
model_data = data.loc[data["persuader_type"] == "human"].copy()

# Ensure 'scenario' is of type string for proper handling in formulas
model_data["scenario"] = model_data["scenario"].astype(str)

# Define the fixed effects formula
formula = "success ~ mental_state"

# Define variance components using vc_formulas
vc_formula = {"scenario": "0 + C(scenario)"}

# Fit the model using the from_formula method
model = BinomialBayesMixedGLM.from_formula(
    formula=formula, vc_formulas=vc_formula, data=model_data
)

# Fit the model using variational Bayes method
result = model.fit_vb()

# Output the summary
print(result.summary())

                     Binomial Mixed GLM Results
                       Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------------------
Intercept                 M    -0.8843   0.1059                      
mental_state[T.reveal]    M     0.3572   0.1455                      
scenario                  V    -1.4646   0.3520 0.231   0.114   0.467
Parameter types are mean structure (M) and variance structure (V)
Variance parameters are modeled as log standard deviations


# H3

## Simple GLM

In [11]:
chance_prob = 0.1
chance_logodds = logit(chance_prob)

model_data = data.loc[
    (data["persuader_type"] == "o1-preview-2024-09-12")
    & (data["mental_state"] == "no_reveal")
].copy()

model_data["offset_term"] = chance_logodds

# Fit the GLM with offset
model = smf.glm(
    formula="success ~ 1",
    data=model_data,
    family=sm.families.Binomial(),
    offset=model_data["offset_term"],
)

result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                  200
Model:                            GLM   Df Residuals:                      199
Model Family:                Binomial   Df Model:                            0
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -92.745
Date:                Fri, 31 Jan 2025   Deviance:                       185.49
Time:                        08:22:37   Pearson chi2:                     200.
No. Iterations:                     4   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6466      0.186      3.475      0.0

/Users/jared/workspace/mindgames/env-mindgames/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/Users/jared/workspace/mindgames/env-mindgames/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: divide by zero encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
/Users/jared/workspace/mindgames/env-mindgames/lib/python3.11/site-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: invalid value encountered in multiply
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +


## Mixed Effects Model

In [12]:
# Ensure 'scenario' is of type string
model_data["scenario"] = model_data["scenario"].astype(str)

# Define the fixed effects formula
formula = "success ~ 1"

# Define variance components
vc_formula = {"scenario": "0 + C(scenario)"}

# Fit the model
model = BinomialBayesMixedGLM.from_formula(
    formula=formula, vc_formulas=vc_formula, data=model_data
)

result = model.fit_vb()
print(result.summary())

               Binomial Mixed GLM Results
          Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
--------------------------------------------------------
Intercept    M    -1.6404   0.1905                      
scenario     V    -0.3076   0.3103 0.735   0.395   1.368
Parameter types are mean structure (M) and variance
structure (V)
Variance parameters are modeled as log standard
deviations


# H4

## Simple GLM

In [13]:
model_data = data.loc[data["persuader_type"] == "o1-preview-2024-09-12"]

model = smf.glm(
    formula="success ~ mental_state", data=model_data, family=sm.families.Binomial()
)

result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                  400
Model:                            GLM   Df Residuals:                      398
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -194.20
Date:                Fri, 31 Jan 2025   Deviance:                       388.39
Time:                        08:22:37   Pearson chi2:                     400.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.3393
Covariance Type:            nonrobust                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

## Mixed Effects Model

In [14]:
# Ensure 'scenario' is of type string
model_data["scenario"] = model_data["scenario"].astype(str)

# Define the fixed effects formula
formula = "success ~ mental_state"

# Define variance components
vc_formula = {"scenario": "0 + C(scenario)"}

# Fit the model
model = BinomialBayesMixedGLM.from_formula(
    formula=formula, vc_formulas=vc_formula, data=model_data
)

result = model.fit_vb()
print(result.summary())

                     Binomial Mixed GLM Results
                       Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------------------
Intercept                 M    -1.5492   0.1273                      
mental_state[T.reveal]    M     2.9550   0.1752                      
scenario                  V    -0.7688   0.3251 0.464   0.242   0.888
Parameter types are mean structure (M) and variance structure (V)
Variance parameters are modeled as log standard deviations


/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_50401/2457102128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data["scenario"] = model_data["scenario"].astype(str)


# H5

## Simple GLM

In [15]:
model_data = data.loc[data["mental_state"] == "no_reveal"]

model = smf.glm(
    formula="success ~ persuader_type", data=model_data, family=sm.families.Binomial()
)

result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                  602
Model:                            GLM   Df Residuals:                      599
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -275.26
Date:                Fri, 31 Jan 2025   Deviance:                       550.52
Time:                        08:22:38   Pearson chi2:                     602.
No. Iterations:                     5   Pseudo R-squ. (CS):            0.04533
Covariance Type:            nonrobust                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

## Mixed Effects Model

In [16]:
# Ensure 'scenario' is of type string
model_data["scenario"] = model_data["scenario"].astype(str)

# Define the fixed effects formula
formula = "success ~ mental_state"

# Define variance components
vc_formula = {"scenario": "0 + C(scenario)"}

# Fit the model
model = BinomialBayesMixedGLM.from_formula(
    formula=formula, vc_formulas=vc_formula, data=model_data
)

result = model.fit_vb()
print(result.summary())

/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_50401/2457102128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data["scenario"] = model_data["scenario"].astype(str)


                     Binomial Mixed GLM Results
                       Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------------------
Intercept                 M    -1.4885   0.1045                      
mental_state[T.reveal]    M     0.0000   2.0000                      
scenario                  V    -1.1738   0.3400 0.309   0.157   0.610
Parameter types are mean structure (M) and variance structure (V)
Variance parameters are modeled as log standard deviations


# H6

## Simple GLM

In [17]:
model_data = data.loc[data["mental_state"] == "reveal"]

model = smf.glm(
    formula="success ~ persuader_type", data=model_data, family=sm.families.Binomial()
)

result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                  602
Model:                            GLM   Df Residuals:                      599
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -327.44
Date:                Fri, 31 Jan 2025   Deviance:                       654.89
Time:                        08:22:38   Pearson chi2:                     602.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.2496
Covariance Type:            nonrobust                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

## Mixed Effects Model

In [18]:
# Ensure 'scenario' is of type string
model_data["scenario"] = model_data["scenario"].astype(str)

# Define the fixed effects formula
formula = "success ~ persuader_type"

# Define variance components
vc_formula = {"scenario": "0 + C(scenario)"}

# Fit the model
model = BinomialBayesMixedGLM.from_formula(
    formula=formula, vc_formulas=vc_formula, data=model_data
)

result = model.fit_vb()
print(result.summary())

/var/folders/y6/5z6f2swj2zqf2122ywg2lfg40000gn/T/ipykernel_50401/1132502827.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_data["scenario"] = model_data["scenario"].astype(str)


                              Binomial Mixed GLM Results
                                        Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
--------------------------------------------------------------------------------------
Intercept                                  M    -1.5127   0.0954                      
persuader_type[T.human]                    M     0.9816   0.1456                      
persuader_type[T.o1-preview-2024-09-12]    M     2.8733   0.1737                      
scenario                                   V    -1.4750   0.3525 0.229   0.113   0.463
Parameter types are mean structure (M) and variance structure (V)
Variance parameters are modeled as log standard deviations


# H7

## Simple GLM

In [19]:
model_data = data

model = smf.glm(
    formula="success ~ persuader_type * mental_state",
    data=model_data,
    family=sm.families.Binomial(),
)

result = model.fit()
print(result.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                success   No. Observations:                 1204
Model:                            GLM   Df Residuals:                     1198
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -602.70
Date:                Fri, 31 Jan 2025   Deviance:                       1205.4
Time:                        08:22:38   Pearson chi2:                 1.20e+03
No. Iterations:                     5   Pseudo R-squ. (CS):             0.2190
Covariance Type:            nonrobust                                         
                                                                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

## Mixed Effects Model

In [20]:
# Ensure categorical variables are properly formatted
model_data["scenario"] = model_data["scenario"].astype(str)
model_data["persuader_type"] = model_data["persuader_type"].astype(str)
model_data["mental_state"] = model_data["mental_state"].astype(str)

# Define the fixed effects formula with interaction
formula = "success ~ persuader_type * mental_state"

# Define variance components
vc_formula = {"scenario": "0 + C(scenario)"}

# Fit the model
model = BinomialBayesMixedGLM.from_formula(
    formula=formula, vc_formulas=vc_formula, data=model_data
)

result = model.fit_vb()
print(result.summary())

                                         Binomial Mixed GLM Results
                                                               Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
-------------------------------------------------------------------------------------------------------------
Intercept                                                         M    -2.2705   0.0712                      
persuader_type[T.human]                                           M     1.3679   0.1060                      
persuader_type[T.o1-preview-2024-09-12]                           M     0.7240   0.1268                      
mental_state[T.reveal]                                            M     0.7233   0.0957                      
persuader_type[T.human]:mental_state[T.reveal]                    M    -0.3436   0.1455                      
persuader_type[T.o1-preview-2024-09-12]:mental_state[T.reveal]    M     2.1919   0.1741                      
scenario                                            